In [1]:
# MODULES
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
import scipy.linalg
import scipy.special
import scipy.stats
from math import ceil
#from statistics import covariance
pi = np.pi

In [2]:
def covariance(a, b):

    if len(a) != len(b):
        return

    a_mean = np.mean(a)
    b_mean = np.mean(b)

    sum = 0

    for i in range(0, len(a)):
        sum += ((a[i] - a_mean) * (b[i] - b_mean))

    return sum/(len(a)-1)

In [3]:
# HYPERPARAMETERS
cs = [ 8/16, 1/16, 5/16, 2/16 ] #ratio of each class
c2 = cs[-1] + cs[-2]
c1 = cs[0] + cs[1]
k = len(cs)
n = 1024 * 2
n_test = 1024 * 2
p = 1024 * 2
gamma = 1
xip = c2 - c1
hs = 500
ins = 500
xi_loop = list(np.linspace(-1, xip -abs(xip), hs)) + list(np.linspace(xip -abs(xip), xip +abs(xip), ins )) + list(np.linspace(xip +abs(xip), 1, hs))

In [4]:
# CLASSES
# Y = 0 <=> Y = -1

cm10 = cs[0]
cm11 = cs[1]
c10 = cs[2]
c11 = cs[3]

cm1 = cm10 + cm11
c1 = c10 + c11

am1 = cs[0] + cs[2]
a1 = cs[1] + cs[3]

e1 = (np.zeros((1, p))).T
e1[0,:] = 1
e2 = (np.zeros((1, p))).T
e2[1,:] = 1


mum10 = - 2 * e1 / cm10
mum11 = - 2 * e2 / cm11
mu10 =  2 * e1 / c10
mu11 =  2 * e2 / c11



cov = np.eye(p, dtype = float)

In [5]:
X = np.zeros( (p,n) )
X_test = np.zeros( (p, n_test) )


for i in range(k):
    # class 1
    if i < 2:
        # majority
        if i == 0:
        #print(mu_c1.shape)
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] = cov@np.random.randn(p, int(cs[i] * n)) + mum10
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =  cov@np.random.randn(p, int(cs[i] * n_test)) + mum10
                    
        # minority
        else:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] =  cov@np.random.randn(p, int(cs[i] * n)) + mum11
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] = cov@np.random.randn(p, int(cs[i] * n_test)) + mum11
            
            # class 2
    else:
                # majority
        if i == 2:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] =   cov@np.random.randn(p, int(cs[i] * n)) + mu10
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] =   cov@np.random.randn(p, int(cs[i] * n_test)) + mu10
                # minority
        else:
            X[: , int(np.sum(cs[:i])*n):int(np.sum(cs[:i+1])*n)] = cov@np.random.randn(p, int(cs[i] * n)) + mu11
            X_test[:,int(np.sum(cs[:i])*n_test):int(np.sum(cs[:i+1]*n_test))] = cov@np.random.randn(p, int(cs[i] * n_test)) + mu11 

y = np.concatenate([-np.ones(int((cs[0] + cs[1]) * n)) / int((cs[0] + cs[1]) * n),np.ones(int((cs[2] + cs[3]) * n)) / int((cs[2] + cs[3]) * n)])
A = np.concatenate([np.zeros(int(cs[0] * n)) / int((cs[0]+cs[2])*n), np.ones(int(cs[1] * n)) / int((cs[1]+cs[3])*n), np.zeros(int(cs[2] * n)) / int((cs[0]+cs[2])*n), np.ones(int(cs[3] * n)) / int((cs[1]+cs[3])*n)])
A_test = np.concatenate([np.zeros(int(cs[0] * n_test)) / int((cs[0]+cs[2])*n_test), np.ones(int(cs[1] * n_test)) / int((cs[1]+cs[3])*n_test), np.zeros(int(cs[2] * n_test)) / int((cs[0]+cs[2])*n_test), np.ones(int(cs[3] * n_test)) / int((cs[1]+cs[3])*n_test)])


XA = np.zeros((p + 1,n), dtype = float)
XA[:-1, :] = X
XA[-1,:] = A            
XTA = np.zeros((p + 1,n_test), dtype = float)
XTA[:-1, :] = X_test
XTA[-1,:] = A_test 
            

In [6]:
#compute of optimal predictor (potentially discriminatory)
# compute R^
sigmaxaxa = np.zeros((p + 1,p  + 1), dtype = float)
for j in range(n):
    xa = np.reshape((XA[:, j]), (p+1, 1))
    #print(xa.shape, xa)
    #print((xa@(xa.T)).shape)
    sigmaxaxa += np.dot(xa,(xa.T))
sigmaxaxa /= n
sigmaxaxam1 = np.linalg.inv(sigmaxaxa)

sigmaxay = np.zeros((p + 1, 1), dtype = float)
for j in range(n):
    data = np.reshape((XA[:, j]), (p + 1, 1))
    #print(data.shape)
    #print(y[j].shape)
    sigmaxay += data * y[j]
sigmaxay /= n
w_chapeau = np.dot(sigmaxaxam1,sigmaxay)
#print(w_chapeau.shape)
LR_chap = np.zeros((n,1), dtype = float)
for j in range(n):
    xa = np.reshape((XA[:, j]), (p+1, 1))
    LR_chap[j,:] = np.dot(w_chapeau.T,xa)
print(LR_chap.shape)
yarray = np.reshape(y, (n, 1))
print(yarray.shape)
aarray = np.reshape(A, (n, 1))
print(aarray.shape)
sigmarcy = covariance(list(LR_chap), y)
sigmaya = covariance(y,A)
sigmaa2 = covariance(A,A)
sigmay2 = covariance(y,y)

alpha = (sigmaya - sigmarcy * sigmaya / sigmay2) / (sigmaa2 - 2 * (sigmaya)**2 / sigmay2 + sigmarcy * (sigmaya)**2 / (sigmay2)**2)

(2048, 1)
(2048, 1)
(2048, 1)


In [7]:
# compute of scores
store_error = np.zeros((len(xi_loop), 8), dtype = float)
g_test = np.zeros( n_test, dtype = float)
gd_test = np.zeros( n_test, dtype = float)
for iter,xi in enumerate(xi_loop):
    g_test = np.zeros( n_test, dtype = float)
    for j in range(n_test):
        xt = np.reshape(XTA[:, j], (p + 1, 1))
        R_ct = np.dot(w_chapeau.T, xt)
        A = xt[ - 1,:]
        R_star = R_ct - alpha*(A - R_ct * sigmaya / sigmay2)
        #print(R_star)
        g_test[j] = float(R_star)
    
    FAR_m = float(np.sum(g_test[  int(cs[0] * n_test) :  int(sum(cs[:2]) * n_test)] > xi)) / float(ceil(cs[1] * n_test))
    CDR_m = float( np.sum( g_test[ int((sum(cs[: - 1]) * n_test)) : ] > xi)) / float(ceil(cs[3] * n_test))
        
    FAR_M = float( np.sum(  g_test[  : int(cs[0] * n_test)  ] > xi)) /  float(int(cs[0] * n_test))
    CDR_M = float( np.sum( g_test[ int(sum(cs[:2]) * n_test) : int(sum(cs[:3]) * n_test)] > xi)) / float(int(cs[2] * n_test))

    FAR_md = float(np.sum(gd_test[  int(cs[0] * n_test) :  int(sum(cs[:2]) * n_test)] > xi)) / float(ceil(cs[1] * n_test))
    CDR_md = float( np.sum( gd_test[ int((sum(cs[: - 1]) * n_test)) : ] > xi)) / float(ceil(cs[3] * n_test))
        
    FAR_Md = float( np.sum(  gd_test[  : int(cs[0] * n_test)  ] > xi)) /  float(int(cs[0] * n_test))
    CDR_Md = float( np.sum( gd_test[ int(sum(cs[:2]) * n_test) : int(sum(cs[:3]) * n_test)] > xi)) / float(int(cs[2] * n_test))

    
    
    store_error[iter, 0] = FAR_m
    store_error[iter, 1] = CDR_m
    store_error[iter, 2] = FAR_M
    store_error[iter, 3] = CDR_M
    store_error[iter, 4] = FAR_md
    store_error[iter, 5] = CDR_md
    store_error[iter, 6] = FAR_Md
    store_error[iter, 7] = CDR_Md
        

def find_xi(labs, lord):
  imin = 0
  d2min = labs[0]**2 + (lord[0] - 1)**2
  for i in range(1, len(labs)):
    dist2 = labs[i]**2 + (lord[1] - 1)**2
    if dist2 < d2min:
      imin = i
      d2min = dist2
  return imin

# get indices of empirical xi
iminority = find_xi(store_error[ : , 0], store_error[ : , 1])
imajority = find_xi(store_error[ : , 2], store_error[ : , 3])
xi_theoric = (cs[2]+cs[3]) - (cs[1] + cs[0]) 
print("xi theorique = ", xi_theoric )
print("xi minorite = ", xi_loop[iminority])
print("xi majorite = ", xi_loop[imajority])
        
        
        

#print(store_error)
# DISPLAY ROC CURVE 
#plt.errorbar(xi_loop, store_error[:,0], store_error[:,1])
#plt.axvline(cs[1]-cs[0],ls='--',color='m')
#plt.xlabel(r'Decision threshold $\xi$')
#plt.ylabel('Misclassification rate')
%matplotlib qt
plt.plot(store_error[ : , 0 ], store_error[ : , 1 ], 'b', linestyle = '-')
plt.plot(store_error[ : , 2], store_error[ : , 3], 'red', linestyle = '-')
plt.plot(store_error[ : , 4 ], store_error[ : , 5 ], 'cyan', linestyle = 'dashed')
plt.plot(store_error[ : , 6 ], store_error[ : , 7 ], 'orange', linestyle = 'dashdot')
plt.title("Woordworth - Gaussian Mixture")
plt.legend(["minorité ND ", "majorité ND ", "minorité D ", "majorité D "])
plt.xlabel('False Alarm Rate')
plt.ylabel('Correct Decision Rate')
plt.show()        

xi theorique =  -0.125
xi minorite =  0.03206412825651302
xi majorite =  0.04008016032064128
